### Test case LVV-T89: Verify implementation of Calibration Image Provenance

Verify that the DMS records the required provenance information for the Calibration Data Products.

**Specification:** For each Calibration Production data product, DMS shall record: the list of input exposures and the range of dates over which they were obtained; the processing parameters; the calibration products used to derive it; and a set of metadata attributes including at least: the date of creation; the calibration image type (e.g. dome flat, superflat, bias, etc); the provenance of the processing software; and the instrument configuration including the filter in use, if applicable.

In the following, we check for all of the listed attributes and information corresponding to a bias frame and a flat frame from the period of the ComCam on-sky campaign in late 2024.

In [1]:
from lsst.daf.butler import Butler

Initialize the butler, and define the collection (corresponding to w_2025_10 processing) we will use.

In [2]:
# Set up standard parameters so we can get to the data.
INSTRUMENT = "LSSTComCam"
COLLECTION = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
butler = Butler("/repo/main")

### Examine biases

Query the butler for `bias` images, then select a single dataId and retrieve the image and its associated metadata.

In [3]:
query = butler.query_datasets('bias', collections=COLLECTION)
bias = butler.get(query[0])
bias_metadata = bias.getMetadata()

Confirm that the bias frame **inputs** are recorded:

In [4]:
for key in bias_metadata.keys():
    if 'CPP_INPUT' in key:
        print(key, ':', bias_metadata[key])

CPP_INPUT_0 : 2024120600427
CPP_INPUT_DATE_0 : 2024-12-07T11:16:39.210
CPP_INPUT_EXPT_0 : 0.0
CPP_INPUT_SCALE_0 : 1.0
CPP_INPUT_1 : 2024120600428
CPP_INPUT_DATE_1 : 2024-12-07T11:16:41.719
CPP_INPUT_EXPT_1 : 0.0
CPP_INPUT_SCALE_1 : 1.0
CPP_INPUT_2 : 2024120600429
CPP_INPUT_DATE_2 : 2024-12-07T11:16:44.228
CPP_INPUT_EXPT_2 : 0.0
CPP_INPUT_SCALE_2 : 1.0
CPP_INPUT_3 : 2024120600430
CPP_INPUT_DATE_3 : 2024-12-07T11:16:46.735
CPP_INPUT_EXPT_3 : 0.0
CPP_INPUT_SCALE_3 : 1.0
CPP_INPUT_4 : 2024120600431
CPP_INPUT_DATE_4 : 2024-12-07T11:16:49.253
CPP_INPUT_EXPT_4 : 0.0
CPP_INPUT_SCALE_4 : 1.0
CPP_INPUT_5 : 2024120600432
CPP_INPUT_DATE_5 : 2024-12-07T11:16:51.773
CPP_INPUT_EXPT_5 : 0.0
CPP_INPUT_SCALE_5 : 1.0
CPP_INPUT_6 : 2024120600433
CPP_INPUT_DATE_6 : 2024-12-07T11:16:54.292
CPP_INPUT_EXPT_6 : 0.0
CPP_INPUT_SCALE_6 : 1.0
CPP_INPUT_7 : 2024120600434
CPP_INPUT_DATE_7 : 2024-12-07T11:16:56.798
CPP_INPUT_EXPT_7 : 0.0
CPP_INPUT_SCALE_7 : 1.0
CPP_INPUT_8 : 2024120600435
CPP_INPUT_DATE_8 : 2024-12-0

Extract the **date range** of the input bias exposures:

In [5]:
for key in bias_metadata.keys():
    if ('DATE-BEG' in key) or ('DATE-END' in key):
        print(key, ':', bias_metadata[key])

for key in bias_metadata.keys():
    if ('MJD-BEG' in key) or ('MJD-END' in key):
        print(key, ':', bias_metadata[key])


SIMULATED DATE-END : None
DATE-BEG : 2024-12-07T11:16:39.210
DATE-END : 2024-12-09T11:47:23.801
MJD-BEG : 60651.4698982639
MJD-END : 60653.4912477024


Look up the **date the combined bias was created**:

In [6]:
for key in bias_metadata.keys():
    if 'CREAT' in key:
        print(key, ':', bias_metadata[key])

CALIB_CREATION_DATETIME : 2025-02-08T19:06:21
CALIB_CREATION_DATE : 2025-02-08
CALIB_CREATION_TIME : 11:06:21 PST


Confirm that the metadata records that this image is a BIAS:

In [7]:
bias_metadata['IMGTYPE']

'BIAS'

Look up the **additional calibration products** that went into this bias's creation:

In [8]:
for key in bias_metadata.keys():
    if ('LSST CALIB RUN' in key) or ('LSST CALIB UUID' in key) or ('LSST CALIB DATE' in key):
        print(key, ':', bias_metadata[key])

LSST CALIB RUN CAMERA : LSSTComCam/calib/DM-48650/unbounded
LSST CALIB UUID CAMERA : 09c852cf-6561-4ce0-881c-259b6a53aedf
LSST CALIB DATE CAMERA : Unknown Unknown
LSST CALIB RUN CCDEXPOSURE : LSSTComCam/raw/all
LSST CALIB DATE CCDEXPOSURE : Unknown Unknown
LSST CALIB RUN CROSSTALK : LSSTComCam/calib/DM-48650/curated/19700101T000000Z
LSST CALIB UUID CROSSTALK : 711f7c84-2597-4788-bd1d-f014fe41568c
LSST CALIB DATE CROSSTALK : Unknown Unknown
LSST CALIB RUN DEFERREDCHARGECALIB : LSSTComCam/calib/DM-48520/DP1/ctiGen.20250207a/20250208T011014Z
LSST CALIB UUID DEFERREDCHARGECALIB : b6063f80-06f5-4de0-89ee-9b29c3871c9a
LSST CALIB DATE DEFERREDCHARGECALIB : Unknown Unknown
LSST CALIB RUN LINEARIZER : LSSTComCam/calib/DM-46360/isrTaskLSST/linearizerGen.20240926a/20240927T182910Z
LSST CALIB UUID LINEARIZER : ff177d36-2a2f-4860-814e-c8052ec93a9f
LSST CALIB DATE LINEARIZER : 2024-09-27 11:29:39.640924
LSST CALIB RUN PTC : LSSTComCam/calib/DM-47447/gainFixup/ptcGen.20241107a/20241107T193934Z
LSST C

#### Retrieve sufficient provenance information to enable recreation of the biases:

First, look up the run collection corresponding to the first input ("INPUT 0") from the bias metadata:

In [9]:
coll = bias_metadata['LSST BUTLER INPUT 0 RUN']
print(coll)

LSSTComCam/calib/DM-48520/DP1/biasGen.20250207a/20250208T185010Z


Extract the **versions of all software packages** that were used for this RUN collection:

In [10]:
bias_isr_packages_refs = butler.query_datasets('packages', collections=coll)

In [11]:
butler.get(bias_isr_packages_refs[0])

Packages({'PIL': '10.3.0', '_libgcc_mutex': '0.1', '_openmp_mutex': '4.5', '_sysroot_linux-64_curr_repodata_hack': '3', 'ads': '0.12.6', 'affine': '2.4.0', 'afw': 'g3fa7bbdb64+5624ebd503', 'aiobotocore': '2.13.1', 'aiohttp': '3.9.5', 'aioinflux': '0.9.0', 'aioitertools': '0.11.0', 'aiosignal': '1.3.1', 'alabaster': '0.7.16', 'alembic': '1.13.2', 'alsa-lib': '1.2.12', 'annotated-types': '0.7.0', 'annotated_types': '0.7.0', 'anyio': '4.4.0', 'aom': '3.9.1', 'apr': '1.7.0', 'archspec': '0.2.3', 'argcomplete': '3.4.0', 'argon2-cffi': '23.1.0', 'argon2-cffi-bindings': '21.2.0', 'arrow': '1.3.0', 'asdf': '3.4.0', 'asdf-astropy': '0.6.1', 'asdf-coordinates-schemas': '0.3.0', 'asdf-standard': '1.1.1', 'asdf-transform-schemas': '0.5.0', 'asdf-wcs-schemas': '0.4.0', 'asdf_astropy': '0.6.1', 'asdf_standard': '1.1.1', 'assist': '1.1.9', 'asteval': '1.0.2', 'astro_metadata_translator': 'g46c833a082+123ace3005', 'astropy': '6.1.2', 'astropy-iers-data': '0.2024.8.5.0.32.23', 'astropy_iers_data': '0.2

Look up the **configurations** that were used in the `cpBiasIsr` and `cpBiasCombine` tasks:

In [12]:
butler.get('cpBiasIsr_config', collections=coll)

lsst.ip.isr.isrTaskLSST.IsrTaskLSSTConfig(saveLogOutput=True, expectWcs=False, qa={'saveStats': True, 'flatness': {'meshX': 256, 'meshY': 256, 'doClip': True, 'clipSigma': 3.0, 'nIter': 3}, 'doWriteOss': False, 'doThumbnailOss': False, 'doWriteFlattened': False, 'doThumbnailFlattened': False, 'thumbnailBinning': 4, 'thumbnailStdev': 3.0, 'thumbnailRange': 5.0, 'thumbnailQ': 20.0, 'thumbnailSatBorder': 2}, doHeaderProvenance=True, doRaiseOnCalibMismatch=False, cameraKeywordsToCompare=[], doDiffNonLinearCorrection=False, doBootstrap=False, doCheckUnprocessableData=True, overscanCamera={'detectorRules': {}, 'defaultDetectorConfig': {'ampRules': {}, 'defaultAmpConfig': {'doSerialOverscan': True, 'serialOverscanConfig': {'fitType': 'MEDIAN_PER_ROW', 'order': 1, 'numSigmaClip': 3.0, 'maskPlanes': ['BAD', 'SAT'], 'overscanIsInt': False, 'maxDeviation': 1000.0, 'doAbsoluteMaxDeviation': True, 'leadingToSkip': 3, 'trailingToSkip': 3}, 'doParallelOverscanCrosstalk': True, 'doParallelOverscan': T

In [13]:
butler.get('cpBiasCombine_config', collections=coll)

lsst.cp.pipe.cpCombine.CalibCombineConfig(saveLogOutput=True, calibrationType='bias', exposureScaling='Unity', scalingLevel='DETECTOR', maxVisitsToCalcErrorFromInputVariance=5, subregionSize=[10000, 200], doVignette=False, doVignetteMask=False, vignette={'xCenter': 0.0, 'yCenter': 0.0, 'radius': 100.0, 'numPolygonPoints': 100}, distributionPercentiles=[0.0, 5.0, 16.0, 50.0, 84.0, 95.0, 100.0], mask=['DETECTED', 'INTRP'], combine='MEANCLIP', clip=3.0, nIter=3, noGoodPixelsMask='BAD', checkNoData=True, censorMaskPlanes=True, stats={'stat': 'MEANCLIP', 'clip': 3.0, 'nIter': 3, 'mask': ['DETECTED', 'BAD', 'NO_DATA']}, connections={'inputExpHandles': 'cpBiasIsrExp', 'inputScales': 'cpScales', 'outputData': 'bias'})

If needed, the **task metadata** can also be retrieved for each of these tasks.

Note also that, while not demonstrated here, the processing logs are retrievable from the Butler, with datasetType names such as `cpBiasIsr_log`.

In [14]:
bias_isr_metadata_refs = butler.query_datasets('cpBiasIsr_metadata', collections=coll)

In [15]:
md = butler.get(bias_isr_metadata_refs[0])

In [16]:
md

TaskMetadata(scalars={}, arrays={}, metadata={'cpBiasIsr': TaskMetadata(scalars={'NUMNANS': 0.0}, arrays={}, metadata={}), 'cpBiasIsr:assembleCcd': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:deferredChargeCorrection': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:crosstalk': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:masking': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:isrStats': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:ampOffset': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:ampOffset:background': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:ampOffset:detection': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:ampOffset:detection:tempLocalBackground': TaskMetadata(scalars={}, arrays={}, metadata={}), 'cpBiasIsr:binning': TaskMetadata(scalars={}, arrays={}, metadata={}), 'quantum': TaskMetadata(scalars={'__version__': 1.0, 'caveats': 0.0}, arrays={'prepU

### Flats

The same can be done for other types of calibrations (e.g., darks, flats, skyflats, crosstalk matrices, etc.). Here, we demonstrate retrieval of the required information for a `flat`.

In [17]:
query_flat = butler.query_datasets('flat', collections=COLLECTION, band='i')
flat = butler.get(query_flat[3])
flat_metadata = flat.getMetadata()

Unlike biases, for flats it is important to know what filter they were obtained with:

In [18]:
for key in flat_metadata.keys():
    if 'FILT' in key:
        print(key, ':', flat_metadata[key])

FILTBAND : i
FILTER : i_06
FILTPOS : 3813.05
FILTSLOT : 1
LSST BUTLER DATAID PHYSICAL_FILTER : i_06


Confirm that the **input flat frames** are recorded:

In [19]:
for key in flat_metadata.keys():
    if 'CPP_INPUT' in key:
        print(key, ':', flat_metadata[key])

CPP_INPUT_0 : 2024112000041
CPP_INPUT_DATE_0 : 2024-11-20T23:45:22.964
CPP_INPUT_EXPT_0 : 0.1
CPP_INPUT_SCALE_0 : 23169.4632076082
CPP_INPUT_1 : 2024112000042
CPP_INPUT_DATE_1 : 2024-11-20T23:46:23.767
CPP_INPUT_EXPT_1 : 0.1
CPP_INPUT_SCALE_1 : 17133.035965747
CPP_INPUT_2 : 2024112000043
CPP_INPUT_DATE_2 : 2024-11-20T23:47:25.565
CPP_INPUT_EXPT_2 : 0.1
CPP_INPUT_SCALE_2 : 13053.9831699614
CPP_INPUT_3 : 2024112000044
CPP_INPUT_DATE_3 : 2024-11-20T23:49:16.716
CPP_INPUT_EXPT_3 : 0.1
CPP_INPUT_SCALE_3 : 8308.69876166517
CPP_INPUT_4 : 2024112000045
CPP_INPUT_DATE_4 : 2024-11-20T23:49:57.792
CPP_INPUT_EXPT_4 : 0.24
CPP_INPUT_SCALE_4 : 17075.6622545633
CPP_INPUT_5 : 2024112000046
CPP_INPUT_DATE_5 : 2024-11-20T23:50:14.422
CPP_INPUT_EXPT_5 : 0.24
CPP_INPUT_SCALE_5 : 16053.5678407903
CPP_INPUT_6 : 2024112000047
CPP_INPUT_DATE_6 : 2024-11-20T23:50:33.846
CPP_INPUT_EXPT_6 : 0.24
CPP_INPUT_SCALE_6 : 14924.1203910307
CPP_INPUT_7 : 2024112000048
CPP_INPUT_DATE_7 : 2024-11-20T23:50:51.296
CPP_INPUT_

Extract the date range spanned by the input flat exposures:

In [20]:
for key in flat_metadata.keys():
    if ('DATE-BEG' in key) or ('DATE-END' in key):
        print(key, ':', flat_metadata[key])

for key in flat_metadata.keys():
    if ('MJD-BEG' in key) or ('MJD-END' in key):
        print(key, ':', flat_metadata[key])


SIMULATED DATE-END : None
DATE-BEG : 2024-11-20T23:45:22.914
DATE-END : 2024-11-20T23:56:39.457
MJD-BEG : 60634.9898485417
MJD-END : 60634.9976789061


Look up additional calibration products that went into the flat:

In [21]:
for key in flat_metadata.keys():
    if ('LSST CALIB RUN' in key) or ('LSST CALIB UUID' in key) or ('LSST CALIB DATE' in key):
        print(key, ':', flat_metadata[key])

LSST CALIB RUN BFKERNEL : LSSTComCam/calib/DM-46360/isrTaskLSST/bfkGen.20240926a/20240927T200534Z
LSST CALIB UUID BFKERNEL : 789a114c-1d61-4b60-86a6-bbd6b8387c5d
LSST CALIB DATE BFKERNEL : Unknown Unknown
LSST CALIB RUN BIAS : LSSTComCam/calib/DM-48520/DP1/biasGen.20250207a/20250208T185010Z
LSST CALIB UUID BIAS : 3265d617-5bee-404e-b625-f0388022826c
LSST CALIB DATE BIAS : 2025-02-08 11:06:25 PST
LSST CALIB RUN CAMERA : LSSTComCam/calib/DM-48650/unbounded
LSST CALIB UUID CAMERA : 09c852cf-6561-4ce0-881c-259b6a53aedf
LSST CALIB DATE CAMERA : Unknown Unknown
LSST CALIB RUN CCDEXPOSURE : LSSTComCam/raw/all
LSST CALIB DATE CCDEXPOSURE : Unknown Unknown
LSST CALIB RUN CROSSTALK : LSSTComCam/calib/DM-48650/curated/19700101T000000Z
LSST CALIB UUID CROSSTALK : 4ef74624-ebce-4b27-bf03-56f7eebffce3
LSST CALIB DATE CROSSTALK : Unknown Unknown
LSST CALIB RUN DARK : LSSTComCam/calib/DM-48520/DP1/darkGen.20250207a/20250208T210026Z
LSST CALIB UUID DARK : b7b11f84-5f63-4688-a3d2-4a67bb42726d
LSST CALIB

Note that this includes a bias, dark, and brighter-fatter (BFKERNEL) correction, which were not present (because they are not needed) for the bias.

Finally, we show that the "config", "log", and "metadata" for the `cpFlatCombine` task are available.

In [22]:
butler.registry.queryDatasetTypes('cpFlatCombine*')

[DatasetType('cpFlatCombine_config', {}, Config),
 DatasetType('cpFlatCombine_log', {band, instrument, detector, physical_filter}, ButlerLogRecords),
 DatasetType('cpFlatCombine_metadata', {band, instrument, detector, physical_filter}, TaskMetadata)]

## Results

We have demonstrated that all the required provenance, configuration, and metadata are available for calibration products derived from the LSST Science Pipelines. We thus deem the result of this test a *Pass*.